In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

def set_css_in_cell_output():
    display(HTML('''
        <style>
            .jupyter-widgets {color: #d5d5d5 !important;}
            .widget-label {color: #d5d5d5 !important;}
        </style>
    '''))

get_ipython().events.register('pre_run_cell', set_css_in_cell_output)
from IPython.core import ultratb
ultratb.VerboseTB._tb_highlight = "bg:#0D0D0D"

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
import time
from scipy import stats
from tqdm.notebook import tqdm
import json

pd.options.display.max_columns = None

In [ ]:
DATA_ROOT = './data/AAPECS/'  # please contact the authors for access to the data

In [3]:
pheno_df = pd.read_csv(f'{DATA_ROOT}/eod_new_time.csv')
data_root = f'{DATA_ROOT}/raw_video_logs'
data_files = [f for f in listdir(data_root) if isfile(join(data_root, f))]

sub_transcripts = {}
sub_lengths = {}

for file in tqdm(data_files):
    df = pd.read_csv(f'{data_root}/{file}')
    df = df[(df.values[:, -1] != 'NO_ANSWER') & (df.values[:, -1] != 'SKIPPED')]

    dates = [x.replace('/', '_') for x in df['Survey Submitted Date'].values]  # dd_mm_yyyy
    times = [x.replace(':', '-') for x in df['Survey Submitted Time'].values]  # dd_mm_yyyy
    addresses = df.values[:, -1]
    userid = df['User Id'].values
    usernum = file.lower().replace('eod', '').replace('vids', '').replace('videos', '').replace('.csv', '').replace('video', '')
    usernum = int(usernum)
    triggers = [x.replace(' ', '') for x in df['Trigger Type'].values]

    assert np.all(['http' in x for x in addresses])
    assert np.all((df['Trigger Type'].values == "DAILY") | (df['Trigger Type'].values == "DELETED TRIGGER") | (df['Trigger Type'].values == "ONCE"))
    assert np.all([x.split('/')[-1].split('.')][-1] == 'mp4' for x in addresses)
    
    sub_pheno_df = pheno_df[pheno_df.participantID == usernum]
    if sub_pheno_df.shape[0] == 0: continue

    
    for i in range(len(addresses)):
        txt = json.load(open(f'{DATA_ROOT}/transcripts_json/{usernum}/{usernum}_{dates[i]}_{times[i]}_{triggers[i]}_{userid[i]}.json'))['text'].strip()
        
        day, month, year = dates[i].split('_')
        sub = usernum
        if sub not in sub_transcripts:
            sub_transcripts[sub] = []
        sub_transcripts[sub].append(txt)
        
        recording_length = json.load(open(f'{DATA_ROOT}/transcripts_json/{usernum}/{usernum}_{dates[i]}_{times[i]}_{triggers[i]}_{userid[i]}.json'))['segments'][-1]['end']
        if sub not in sub_lengths:
            sub_lengths[sub] = []
        sub_lengths[sub].append(recording_length)


A Jupyter Widget

In [4]:
name_mapping_aapecs = {
    'O': 'neoOpenness',
    'C': 'neoConscientiousness',
    'E': 'neoExtraversion',
    'A': 'neoAgreeableness',
    'N': 'neoNeuroticism'
}
cols_of_interest = sorted(name_mapping_aapecs.values())

In [5]:
scales_df = pd.read_csv(f'{DATA_ROOT}/selfReport.csv')
scales_df = scales_df[scales_df.participantID.isin([int(x) for x in sub_lengths])]
scales_df = scales_df[['participantID']+list(cols_of_interest)]
print(scales_df.shape)
scales_df.head()

(108, 6)


,participantID,neoAgreeableness,neoConscientiousness,neoExtraversion,neoNeuroticism,neoOpenness
0,1,2.750000,2.875000,1.291667,2.875000,1.875000
1,2,2.666667,2.708333,1.750000,2.416667,2.166667
2,3,2.430556,1.083333,2.000000,3.625000,3.166667
3,4,2.250000,2.666667,2.958333,2.833333,3.208333
4,5,3.000000,3.083333,1.250000,2.750000,1.625000


In [6]:
liwc_df = pd.read_csv(f'{DATA_ROOT}/AAPECS_LIWC_2015.csv')
liwc_df['participantID'] = [int(x.split('_')[0]) for x in liwc_df.Filename.values.flatten()]
liwc_df = liwc_df[liwc_df.participantID.isin(scales_df.participantID.values.flatten())]
liwc_df = liwc_df.drop(columns=['Filename', 'Segment'])
liwc_df = liwc_df.groupby('participantID', as_index=False).mean()
print(liwc_df.shape)
liwc_df.head()

(108, 94)


,participantID,WC,Analytic,Clout,Authentic,Tone,WPS,Sixltr,Dic,function,pronoun,ppron,i,we,you,shehe,they,ipron,article,prep,auxverb,adverb,conj,negate,verb,adj,compare,interrog,number,quant,affect,posemo,negemo,anx,anger,sad,social,family,friend,female,male,cogproc,insight,cause,discrep,tentat,certain,differ,percept,see,hear,feel,bio,body,health,sexual,ingest,drives,affiliation,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,relativ,motion,space,time,work,leisure,home,money,relig,death,informal,swear,netspeak,assent,nonflu,filler,AllPunc,Period,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,1,115.294118,41.665294,31.838824,90.496471,44.923529,13.054706,10.347059,96.483529,56.887647,17.508824,12.078235,8.935294,1.250000,0.240000,1.555882,0.096471,5.431176,6.471176,13.371765,10.751176,7.502353,6.569412,2.368824,19.808235,1.897059,0.950588,0.561176,1.455882,3.543529,5.827647,3.428235,2.367059,0.960000,1.077647,0.032941,7.495294,0.307059,1.068235,1.550000,0.595882,8.410000,0.544118,1.154706,0.426471,2.210588,1.131765,3.271176,1.123529,0.444118,0.300588,0.378235,2.788824,0.487059,0.511765,0.000000,1.845882,10.574118,3.183529,1.829412,1.626471,3.975294,0.188824,9.394706,13.085882,2.592941,21.197647,2.634706,6.294118,12.684118,2.077059,1.091765,1.055882,0.697059,0.148824,0.000000,0.675294,0.048235,0.000000,0.530588,0.048235,0.000000,25.293529,9.021765,10.814118,0.0,0.0,0.221176,0.000,0.000000,0.0,5.235882,0.0,0.000000
1,2,144.428571,17.546667,23.048095,88.160952,32.511429,16.728095,12.862381,96.456667,63.140000,22.073810,15.873810,12.687619,0.848571,0.513333,1.414762,0.410000,6.199524,4.570476,13.096667,11.674286,8.927143,7.966190,2.596190,20.600000,4.989048,3.267143,1.276667,0.543810,4.094762,6.098095,2.837143,3.009524,0.806190,1.159524,0.456667,8.583333,1.561429,0.223810,2.293810,0.335238,14.391429,2.222857,1.304286,1.163333,4.281905,1.403810,4.709048,2.251905,0.487143,0.480952,1.283810,2.055238,0.692857,1.053333,0.032381,0.267143,7.240952,2.692857,1.540000,1.424762,1.776190,0.186667,7.179524,11.140000,1.571905,13.760476,2.495714,5.749048,5.657619,0.935238,0.607619,0.626667,0.443333,0.159048,0.000000,0.827619,0.092381,0.028571,0.257143,0.449524,0.000000,19.088095,7.040952,7.640476,0.0,0.0,0.100000,0.000,0.000000,0.0,4.305714,0.0,0.000000
2,3,61.333333,18.160000,10.781667,97.015000,37.816667,12.678333,12.818333,94.311667,59.888333,20.030000,15.513333,14.793333,0.173333,0.000000,0.546667,0.000000,4.515000,5.268333,9.700000,9.071667,8.658333,10.750000,1.715000,16.856667,6.563333,3.701667,1.076667,0.498333,3.915000,9.091667,4.406667,4.396667,2.536667,0.575000,0.535000,6.370000,0.273333,1.498333,0.820000,0.000000,15.950000,3.541667,0.793333,0.421667,5.488333,0.535000,5.743333,2.821667,0.000000,0.000000,2.821667,1.306667,0.248333,0.771667,0.000000,0.286667,8.565000,2.941667,1.093333,1.825000,2.698333,0.500000,9.226667,7.095000,1.590000,12.855000,1.998333,3.923333,7.218333,1.475000,1.796667,0.286667,0.000000,0.000000,0.000000,0.671667,0.248333,0.173333,0.421667,0.000000,0.000000,17.258333,8.501667,6.230000,0.0,0.0,0.000000,0.000,0.000000,0.0,2.523333,0.0,0.000000
3,4,166.944444,10.802222,22.613333,76.110556,45.163889,21.277778,13.885556,94.066667,60.402778,20.606111,12.670556,9.758889,0.617222,0.291667,1.768889,0.235556,7.934444,4.333333,10.147222,11.880000,9.051111,11.586667,1.803889,20.716667,5.265556,2.775000,1.432778,0.751667,2.394444,8.500556,4.387222,3.960000,1.343889,0.763889,0.976667,6.962778,0.000000,0.807222,0.061111,2.455000,15.148333,3.295556,1.787778,1.181111,4.110000,1.712222,4.149444,2.676667,0.640000,0.468333,1.453889,2.243333,0.666667,0.768889,0.412778,0.408889,5.270000,1.348333,1.171111,0.848889,1.639444,0.542222,6.353333,12.808333,1.840556,12.783889,2.033889,4.668889,6.073889,1.285000,1.127778,0.515000,0.159444,0.041667,0.087222,2.236111,0.533889,0.000000,0.422222,0.846667,0.116111,28.240000,7.569444,14.231111,0.0,0.0,0.000000,0.105,0.305000,0.0,5.999444,0.0,0.027778
4,5,87.789474,44.47

In [7]:
from sklearn.model_selection import KFold
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, sem

def fit_ols_models_cv(scales_df, liwc_df, outer_splits=5, random_state=42):
    """
    Perform k-fold cross-validation to compute multiple correlation (R)
    between LIWC predictors and each trait.

    Parameters:
        scales_df: DataFrame with subject and trait columns.
        liwc_df: DataFrame with subject and LIWC features.
        outer_splits: Number of outer CV folds.
    
    Returns:
        final_r: dict of mean and std_err cross-validated R per trait.
    """
    merged = pd.merge(scales_df, liwc_df, on='subject').dropna()
    assert merged.shape[0] == 60

    predictor_cols = [col for col in liwc_df.columns if col != 'subject']
    X_all = merged[predictor_cols].values
    y_all = {trait: merged[trait].values for trait in scales_df.columns if trait != 'subject'}

    outer_cv = KFold(n_splits=outer_splits, shuffle=True, random_state=random_state)
    avg_cv_r = {trait: [] for trait in y_all}

    for train_idx, test_idx in outer_cv.split(X_all):
        X_train, X_test = X_all[train_idx], X_all[test_idx]

        X_train_w_intercept = sm.add_constant(X_train)
        X_test_w_intercept = sm.add_constant(X_test)

        for trait, y in y_all.items():
            y_train, y_test = y[train_idx], y[test_idx]
            model = sm.OLS(y_train, X_train_w_intercept).fit()
            y_pred = model.predict(X_test_w_intercept)

            r, _ = pearsonr(y_test, y_pred)
            avg_cv_r[trait].append(r)

    final_r = {trait: (np.mean(r_vals), sem(r_vals)) for trait, r_vals in avg_cv_r.items()}
    return final_r

In [11]:
def pretty_print_ols_cv_results(results):
    """
    Pretty prints 5-fold CV results for OLS without PCA.

    Parameters:
        results (dict): Trait name → cross-validated R
    """
    max_len = max(len(trait) for trait in results)
    lines = []
    lines.append("5-Fold Cross-Validation Results (simple OLS)")
    lines.append("Multiple Correlation Coefficient: (trait ~ all LIWC predictors)")
    for trait in results:
        r_ci_upper = results[trait][0]+(1.96*results[trait][1])
        r_ci_lower = results[trait][0]-(1.96*results[trait][1])
        r_ci = f'({r_ci_lower:>5.3f} - {r_ci_upper:.3f})'
        lines.append(f"• {trait:<{max_len}}  R = {results[trait][0]:>5.3f}  R-SE = {results[trait][1]:>5.3f}   R-95%CI = {r_ci}")
    return "\n".join(lines)

cv_r = fit_ols_models_cv(scales_df, liwc_df)
print(pretty_print_ols_cv_results(cv_r))


5-Fold Cross-Validation Results (simple OLS)
Multiple Correlation Coefficient: (trait ~ all LIWC predictors)
• neoAgreeableness      R = -0.063  R-SE = 0.111   R-95%CI = (-0.281 - 0.154)
• neoConscientiousness  R = 0.194  R-SE = 0.028   R-95%CI = (0.140 - 0.249)
• neoExtraversion       R = 0.096  R-SE = 0.119   R-95%CI = (-0.137 - 0.329)
• neoNeuroticism        R = 0.160  R-SE = 0.099   R-95%CI = (-0.034 - 0.354)
• neoOpenness           R = -0.150  R-SE = 0.106   R-95%CI = (-0.359 - 0.058)
